Assumes raw data from dbGAP on <code>/labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation</code> 

In [1]:
import os

basedir = "/labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation";

# Change working directory
os.chdir(basedir)

# Create simbolic link to PED/MAP file
!ln -s ./86679/NHLBI/SEA_Herrington/phs000349v1/p1/genotype/phg000121v1/genotype-calls-matrixfmt/SEA_Phase2.map .
!ln -s ./86679/NHLBI/SEA_Herrington/phs000349v1/p1/genotype/phg000121v1/genotype-calls-matrixfmt/SEA_Phase2.ped .
!ls

86679  BKP  SEA_Phase2.map  SEA_Phase2.ped  TOBEDELETED


The array from Perlegen was design based on human assembly <code>hg18</code>. In order to proceed with data_preparation we need to liftover raw files to <code>hg19</code> (Michigan Imputation Server, HRC reference panel) and <code>hg38</code> (TOPMed Imputation Server and reference panel)

Before liftover, we need to preprocess data to avoid losing data from chromosomes X and Y.

In [2]:
# Make sure we are in the basedir
os.chdir(basedir)

# X and Y chromosomes are coded as 23 and 24 in the original MAP file. Liftovering this way will make SNPs on these 2 chromosomes
# to be dropped from the analysis. Recode them to X and Y to avoid losing this data
!perl -pe "s/^23/X/" SEA_Phase2.map | perl -pe "s/^24/Y/" > SEA_Phase2.sex_recoded.map

# PED file remains the same. Therefore, it's ok to use a symbolic link to reference it
!ln -s SEA_Phase2.ped SEA_Phase2.sex_recoded.ped

# List files
!ls

86679  SEA_Phase2.map  SEA_Phase2.sex_recoded.map  TOBEDELETED
BKP    SEA_Phase2.ped  SEA_Phase2.sex_recoded.ped


The data liftover of MAP/PED files will be done using the tool <code>liftOverPlink</code> (https://github.com/sritchie73/liftOverPlink)

In [3]:
# Assumes liftOverPlink.py script is installed in data_preparation folder as well dependencies properly present 
# (liftOver script from UCSC and chain file)

# Hg19
print("Converting to hg19")
!mkdir hg19_liftover
!../tools/liftOverPlink/liftOverPlink.py \
    -m SEA_Phase2.sex_recoded.map \
    -p SEA_Phase2.sex_recoded.ped \
    -e ../tools/liftOver \
    -c ../aux_files/hg18ToHg19.over.chain.gz \
    -o ./hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover

# Hg38
print("Converting to hg38")
!mkdir hg38_liftover
!../tools/liftOverPlink/liftOverPlink.py \
    -m SEA_Phase2.sex_recoded.map \
    -p SEA_Phase2.sex_recoded.ped \
    -e ../tools/liftOver \
    -c ../aux_files/hg18ToHg38.over.chain.gz \
    -o ./hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover

# List files
print("\nFiles:")
!ls -lha * hg*_liftover/*

Converting to hg19
Converting MAP file to UCSC BED file...
SUCC:  map->bed succ
Lifting BED file...
Reading liftover chains
Mapping coordinates
SUCC:  liftBed succ
Converting lifted BED file back to MAP...
SUCC:  bed->map succ
Updating PED file...
SUCC:  liftPed succ
cleaning up BED files...
Converting to hg38
Converting MAP file to UCSC BED file...
SUCC:  map->bed succ
Lifting BED file...
Reading liftover chains
Mapping coordinates
SUCC:  liftBed succ
Converting lifted BED file back to MAP...
SUCC:  bed->map succ
Updating PED file...
SUCC:  liftPed succ
cleaning up BED files...

Files:
-rw-rw----+ 1 rgsousa scg_lab_tassimes 1.7K Apr 12 18:39 hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.bed.unlifted
-rw-rw----+ 1 rgsousa scg_lab_tassimes 2.4M Apr 12 18:39 hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.map
-rw-rw----+ 1 rgsousa scg_lab_tassimes 433M Apr 12 18:40 hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.ped
-rw-rw----+ 1 rgsousa scg_lab_tassimes 3.2K Apr 12 18:40 hg38

Perfome data preparation steps as instructed in the page from Michigan Imputation Server https://imputationserver.readthedocs.io/en/latest/prepare-your-data/

In [4]:
%%bash

# Load plink module at SCG
module load plink

for input_file_basename in hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover; do

    # Convert ped/map to bed
    plink \
        --file ${input_file_basename} \
        --allow-extra-chr \
        --chr 1-24 \
        --make-bed \
        --out ${input_file_basename}

    # Create a frequency file
    plink \
        --bfile ${input_file_basename} \
        --allow-extra-chr \
        --chr 1-24 \
        --freq \
        --out ${input_file_basename}

done;

PLINK v1.90p 64-bit (30 Nov 2019)              www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.log.
Options in effect:
  --allow-extra-chr
  --chr 1-24
  --file hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover
  --make-bed
  --out hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover

515883 MB RAM detected; reserving 257941 MB for main workspace.
Allocated 14523 MB successfully, after larger attempt(s) failed.
.ped scan complete (for binary autoconversion).111111111111111111112121212121212121212131313131313131313131314141414141414141414141515151515151515151516161616161616161616161717171717171717171717181818181818181818181919191919191919191919202020202020202020202021212121212121212121222222222222222222222223232323232323232323232424242424242424242425252525252525252525252626262626262626262626272727272727272727272728282828282828282828292929292929292929292930303030303

hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.hh ); many commands treat
these as missing.
hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover.hh ); many commands treat
these as missing.
hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover.hh ); many commands treat
these as missing.
hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover.hh ); many commands treat
these as missing.


Run perl script <code>HRC-1000G-check-bim.pl</code>(https://www.well.ox.ac.uk/~wrayner/tools/) to generate input files as described at https://imputationserver.readthedocs.io/en/latest/prepare-your-data/

In [5]:
%%bash

# Load plink module at SCG
module load plink tabix

cd /labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/

# Execute HRC-1000G-check-bim.pl script to prepare data for imputation for hg19
perl ../../tools/HRC-1000G-check-bim.pl \
    -b ./SEA_Phase2.sex_recoded.hg19_liftover.bim \
    -f ./SEA_Phase2.sex_recoded.hg19_liftover.frq \
    -r ../../aux_files/HRC.r1-1.GRCh37.wgs.mac5.sites.tab \
    -o ./ \
    -h
sh ./Run-plink.sh



         Script to check plink .bim files against HRC/1000G for
        strand, id names, positions, alleles, ref/alt assignment
                    William Rayner 2015-2020
                        wrayner@well.ox.ac.uk

                               Version 4.3


Options Set:
Reference Panel:             HRC
Bim filename:                ./SEA_Phase2.sex_recoded.hg19_liftover.bim
Reference filename:          ../../aux_files/HRC.r1-1.GRCh37.wgs.mac5.sites.tab
Allele frequencies filename: ./SEA_Phase2.sex_recoded.hg19_liftover.frq
Plink executable to use:     plink

Chromosome flag set:         No
Allele frequency threshold:  0.2

Path to plink bim file: /oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover
Writing output files to: ./


Reading ../../aux_files/HRC.r1-1.GRCh37.wgs.mac5.sites.tab
2000000
4000000
6000000
8000000
10000000
12000000
14000000
16000000
18000000
20000000
22000000
24000000
26000000
28000000
30000000
32000000
3

/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/TEMP1.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/TEMP2.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/TEMP3.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/TEMP4.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover-updated.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg19_liftover/SEA_Phase2.sex_recoded.hg19_liftover-updated-chr23.hh
); many commands trea

Repeat the process for hg38/TOPMed data

In [6]:
%%bash

# Load plink module at SCG
module load plink tabix

cd /labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/

# Execute HRC-1000G-check-bim.pl script to prepare data for imputation for hg38/TOPMed data (may take some time to run)
perl ../../tools/HRC-1000G-check-bim.pl \
    -b ./SEA_Phase2.sex_recoded.hg38_liftover.bim \
    -f ./SEA_Phase2.sex_recoded.hg38_liftover.frq \
    -r ../../aux_files/PASS.Variantsbravo-dbsnp-all.tab.gz \
    -o ./ \
    -h
sh ./Run-plink.sh



         Script to check plink .bim files against HRC/1000G for
        strand, id names, positions, alleles, ref/alt assignment
                    William Rayner 2015-2020
                        wrayner@well.ox.ac.uk

                               Version 4.3


Options Set:
Reference Panel:             HRC
Bim filename:                ./SEA_Phase2.sex_recoded.hg38_liftover.bim
Reference filename:          ../../aux_files/PASS.Variantsbravo-dbsnp-all.tab.gz
Allele frequencies filename: ./SEA_Phase2.sex_recoded.hg38_liftover.frq
Plink executable to use:     plink

Chromosome flag set:         No
Allele frequency threshold:  0.2

Path to plink bim file: /oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover
Writing output files to: ./


Reading ../../aux_files/PASS.Variantsbravo-dbsnp-all.tab.gz
Reference Panel is zipped
2000000
4000000
6000000
8000000
10000000
12000000
14000000
16000000
18000000
20000000
22000000
24000000
26000000


/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/TEMP1.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/TEMP2.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/TEMP3.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/TEMP4.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover-updated.hh
); many commands treat these as missing.
/oak/stanford/scg/lab_tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/SEA_Phase2.sex_recoded.hg38_liftover-updated-chr23.hh
); many commands trea

Final processing and file compression

In [8]:
%%bash

cd /labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/hg38_liftover/

# When working with hg38, imputation servers expect chromosomes encoded with prefix (e.g. chr20)
# Append chr prefix on VCF files and save them as *.chr_prefix.vcf 
for vcf_file in $(ls *.vcf); do
  
  echo -e "Processing file: ${vcf_file}";
  vcf_file_basename=$(basename ${vcf_file} .vcf);
  output_vcf_file=${vcf_file_basename}.chr_prefix.vcf;
  
  # Write header
  cat ${vcf_file} | grep "^#" | perl -pe "s/contig=<ID=/contig=<ID=chr/" > ${output_vcf_file};
  
  # Write VCF file body
  cat ${vcf_file} | grep -v "^#" | perl -pe "s/^/chr/" >> ${output_vcf_file};
      
  done

Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr10.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr11.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr12.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr13.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr14.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr15.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr16.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr17.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr18.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr19.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr1.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr20.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr21.vcf
Processing file: SEA_Phase2.sex_recoded.hg38_liftover-updated-chr

 Compress output VCFs bzip (required by imputation servers)

In [9]:
%%bash

# Return to basedir
cd /labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/

# Load module at SCG
module load tabix

# Compress files as bzip
for vcf_file in $(ls ./hg*_liftover/*.vcf); do
    bgzip ${vcf_file}; 
    done

Split samples of **Whites** and **Blacks** prior imputation to get more accurate imputation R2 

In [23]:
import pandas as pd

os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/")

# Read phenotypes and recode sex and race attributes
sea_phenotypes = pd.read_table(
    "data_preparation_to_imputation/86679/NHLBI/SEA_Herrington/phs000349v1/p1/phenotype/phs000349.v1.pht002191.v1.p1.c1.SEA_Phase2_Subject_Phenotypes.GRU.txt",
    index_col="seaid",
    comment="#")

# Recode Sex and Race
sea_phenotypes = sea_phenotypes.replace( {
    "sex": { 1:"male", 2:"female" },
    "race": { 1:"white", 2:"black" }
    } )

# Create a file with subject ids of whites and another with blacks
for race in ["white", "black"]:
    subjects_ids = sea_phenotypes[ sea_phenotypes["race"] == race ].index.to_list()
    subjects_ids_in_vcf = [ "_".join( [subjects_ids[i], subjects_ids[i] ] ) for i in range( len(subjects_ids) ) ]
    
    with open( './data_preparation_to_imputation/subject_ids_' + race + 's.txt', 'w') as f:
        for subject_id in subjects_ids_in_vcf:
            f.write("%s\n" % subject_id)

In [32]:
%%bash

# Return to basedir
cd /labs/tassimes/rodrigoguarischi/projects/sea/data_preparation_to_imputation/

# Load module at SCG
module load bcftools tabix

# Loop over VCFs subseting whites and blacks and writing output files
for subject_ids_file in $(ls subject_ids_*); do

    race=$( echo ${subject_ids_file} | cut -d. -f 1 | cut -d_ -f 3 )
    
    for vcf_file in $(ls ./hg*_liftover/*.vcf.gz); do
        
        vcf_basename=$(basename ${vcf_file} .vcf.gz)
        vcf_dirname=$(dirname ${vcf_file})
        output_vcf_file="${vcf_dirname}/${race}/${vcf_basename}_${race}.vcf.gz"
        
        # Create output directory if it doesn't exist (no error, if it does)
        mkdir -p "${vcf_dirname}/${race}"
        
        # Run bcftools to filter VCF by each race
        bcftools view -S ${subject_ids_file} ${vcf_file} | bgzip > ${output_vcf_file}
        
        done
    done
    
# For TOPMed imputation we will start with files based on build 38, so we can remove those VCFs without chr prefix
find ./hg38_liftover/whites/ -type f | grep ".vcf.gz" | grep -v "chr_prefix" | xargs rm
find ./hg38_liftover/blacks/ -type f | grep ".vcf.gz" | grep -v "chr_prefix" | xargs rm